# Develop new Constrained Queueing Models

In [15]:
from gpkit import Model, Variable, units

In [16]:
import numpy as np

In [17]:
from gpx.manufacturing import QNACell, Process, FabLine

## Blocking in cell

Create two cells without queueing in between

In [18]:
processes = [Process(10*units('min')), 
             Process(10*units('min'))]

### Baseline with queueing

In [19]:
m1 = FabLine([QNACell(processes[0]), QNACell(processes[1])])

In [20]:
m1.cost = m1.L/100. + np.sum([c.m for c in m1.cells])

In [21]:
m1.substitutions[m1.lam] = 6
m1.substitutions[processes[0].cv] = 1
m1.substitutions[processes[1].cv] = 1

In [22]:
print(m1.solve().table())

UnboundedGP: QNACell2.Wq has no lower bound.

### No buffer before second cell

$$ W_1 \geq W_{q,1} + \tau_{s,1} + W_{q,2} $$
$$ W_2 \geq \tau_{s,2} $$

- Station with no queueing has to be specified to return the appropriate constraint
- Additional time is added to the previous station

In [23]:
cells = [QNACell(processes[0]), QNACell(processes[1])]

In [24]:
# replace the constraints
c1 = cells[0]
c2 = cells[1]
# cells[0][0] = c1.W >= c1.Wq + c1.tnu + c2.Wq  # need to replace on tnu because that's rate
cells[0][2] = c1.tnu >= c1.process.t + c2.Wq
cells[1][0] = c2.W >= c2.tnu

In [25]:
m2 = FabLine(cells)

In [26]:
m2.cost = m2.L/100. + np.sum([c.m for c in m2.cells])

In [27]:
m2.substitutions[m2.lam] = 6
m2.substitutions[processes[0].cv] = 1
m2.substitutions[processes[1].cv] = 1

In [28]:
print(m2.solve().table())

UnboundedGP: QNACell6.Wq has no lower bound.

QNACell6.\alpha has no lower bound.

QNACell2.Wq has no lower bound.

### Now look at peak performance with fixed cell counts

In [29]:
cells = [QNACell(processes[0]), QNACell(processes[1])]

In [30]:
# replace the constraints
c1 = cells[0]
c2 = cells[1]
# cells[0][0] = c1.W >= c1.Wq + c1.tnu + c2.Wq  # need to replace on tnu because that's rate
cells[0][2] = c1.tnu >= c1.process.t + c2.Wq
cells[1][0] = c2.W >= c2.tnu

In [31]:
m3 = FabLine(cells)

In [32]:
m3.substitutions[processes[0].cv] = 1
m3.substitutions[processes[1].cv] = 1
m3.substitutions.update({
    c1.m : 2,
    c2.m : 1,
    m3.L : 15,
})

In [33]:
m3.cost = 1/m3.lam

In [34]:
print(m3.solve().table())

UnboundedGP: QNACell8.\alpha has no lower bound.

QNACell8.Wq has no lower bound.

QNACell2.Wq has no lower bound.

## Empty Cell is the Only Queue

# Testing the qna cell processing into the fabline with no queueing

In [1]:
from gpx.manufacturing import QNACell, FabLine

In [2]:
test = QNACell(queueing_at_cell=False)

In [3]:
test

<gpkit.QNACell object containing 7 top-level constraint(s) and 15 variable(s)>

In [4]:
cells = [QNACell(), QNACell(queueing_at_cell=False)]

In [5]:
test = FabLine(cells)

In [10]:
cells[1].allows_queueing

False

In [6]:
cells[1].aux_time

[gpkit.Variable(QNACell2.Wq [min])]

In [7]:
print(test)

FabLine

Cost
----
 1

Constraints
-----------
  L = FabLine.\lambda·FabLine.W
  QNACell2.c2a >= QNACell1.c2d
  QNACell2.c2d <= QNACell1.c2a
  FabLine.\lambda = QNACell1.\lambda
  FabLine.\lambda = QNACell2.\lambda
  QNACell1.t_{auxiliary} >= QNACell2.Wq
  QNACell2.t_{auxiliary} >= QNACell2.Wq
  FabLine.W >= QNACell1.W + QNACell2.W

  QNACell1
   QNACell1.Wq >= QNACell1.rho/QNACell1.\alpha·(QNACell1.c2a + QNACell1.c2s)/2·QNACell1.t_{\eta}/QNACell1.m
   QNACell1.c2d >= QNACell1.\alpha·QNACell1.c2a + QNACell1.rho·QNACell1.c2s
   QNACell1.\alpha + QNACell1.rho <= 1
   QNACell1.\lambda <= QNACell1.rho·QNACell1.m/QNACell1.t_{\eta}
   QNACell1.c2s >= cv²·QNACell1.\chi_{cv}²
   QNACell1.W >= QNACell1.Wq + QNACell1.t_{\eta}
   QNACell1.t_{\eta} >= t·QNACell1.\eta_t + QNACell1.t_{auxiliary}

  QNACell2
   QNACell2.Wq >= QNACell2.rho/QNACell2.\alpha·(QNACell2.c2a + QNACell2.c2s)/2·QNACell2.t_{\eta}/QNACell2.m
   QNACell2.c2d >= QNACell2.\alpha·QNACell2.c2a + QNACell2.rho·QNACell2.c2s
   QNACell2

In [14]:
test.substitutions.update({
    cells[0]
})

{QNACell1.\chi_{cv}: 1,
 QNACell1.\eta_t: 1,
 QNACell2.\chi_{cv}: 1,
 QNACell2.\eta_t: 1}

In [11]:
cells[1].aux_time = 0

In [13]:
print(cells[1])

QNACell2

Cost
----
 1

Constraints
-----------
  Wq >= QNACell2.rho/QNACell2.\alpha·(QNACell2.c2a + QNACell2.c2s)/2·QNACell2.t_{\eta}/QNACell2.m
  c2d >= QNACell2.\alpha·QNACell2.c2a + QNACell2.rho·QNACell2.c2s
  QNACell2.\alpha + QNACell2.rho <= 1
  \lambda <= QNACell2.rho·QNACell2.m/QNACell2.t_{\eta}
  c2s >= cv²·QNACell2.\chi_{cv}²
  W >= t_{\eta}
  t_{\eta} >= t·QNACell2.\eta_t + QNACell2.t_{auxiliary}
